# Tema 1 - Conhecer a representação word embedding com Word2Vec

## Word2Vec vs. bag-of-words

In [1]:
from sklearn.feature_extraction.text import CountVectorizer

textos= ["eu sou muito bom",
         "eu sou muito ruim",
         "eu sou excelente",
         "o papa é pop"]

vetorizador = CountVectorizer()

In [2]:
bag_of_words = vetorizador.fit_transform(textos)

In [3]:
bag_of_words.toarray()

array([[1, 1, 0, 1, 0, 0, 0, 1],
       [0, 1, 0, 1, 0, 0, 1, 1],
       [0, 1, 1, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 1, 1, 0, 0]])

In [4]:
vetorizador.get_feature_names()

['bom', 'eu', 'excelente', 'muito', 'papa', 'pop', 'ruim', 'sou']

In [5]:
vetor_excelente = vetorizador.transform(["excelente"])
print(vetor_excelente.toarray())

[[0 0 1 0 0 0 0 0]]


In [6]:
vetor_papa = vetorizador.transform(["papa"])
print(vetor_papa.toarray())

[[0 0 0 0 1 0 0 0]]


In [7]:
from google.colab import drive 
drive.mount("/content/drive")

Mounted at /content/drive


In [8]:
!unzip "/content/drive/My Drive/Unyleya - Mineração de Texto e PLN/cbow_s300.zip"

Archive:  /content/drive/My Drive/Unyleya - Mineração de Texto e PLN/cbow_s300.zip
  inflating: cbow_s300.txt           


In [9]:
with open("cbow_s300.txt") as iteradorArquivo:
  for contadorLinha in range(10):
    print(next(iteradorArquivo))

929606 300

</s> -0.001667 -0.000158 -0.000026 0.001300 -0.000796 0.001527 0.000046 0.000584 0.000449 -0.000100 0.000353 0.001251 0.001069 0.000506 0.000574 0.000838 -0.000930 -0.001220 0.000317 0.001315 -0.001120 0.001373 -0.000040 -0.001580 0.000421 -0.000667 -0.001556 -0.000746 0.001604 0.001157 -0.000027 0.000354 0.000358 -0.000527 -0.000573 -0.001512 -0.001557 -0.001637 0.001617 -0.001511 -0.001022 -0.001426 0.001086 -0.001033 0.000593 0.000724 0.000627 -0.000450 -0.001140 0.000333 0.000524 0.001541 0.000284 0.000617 -0.000807 -0.000088 -0.000364 0.001126 -0.001230 -0.001138 -0.001280 0.001330 0.001257 0.000576 0.000764 0.000684 0.001008 -0.000215 -0.000629 -0.001228 -0.001557 -0.000311 -0.000246 0.000045 0.001136 -0.000645 -0.000549 0.001099 0.000858 -0.000886 0.000553 0.000303 0.001433 0.000732 0.001321 -0.000894 -0.000700 -0.000661 -0.001484 -0.000950 -0.001556 -0.000809 0.000348 -0.000068 0.000724 -0.000569 -0.000161 -0.001628 -0.001437 -0.000259 -0.000296 -0.001571 0.000149 0

## Preparando ambiente para o novo projeto

In [10]:
import pandas as pd

In [11]:
!unzip "/content/drive/My Drive/Unyleya - Mineração de Texto e PLN/noticias_dataset.zip"

Archive:  /content/drive/My Drive/Unyleya - Mineração de Texto e PLN/noticias_dataset.zip
  inflating: noticias_dataset.csv    


In [12]:
noticias_dataset = pd.read_csv("noticias_dataset.csv")

In [13]:
noticias_dataset.head()

,title,text,date,category,subcategory,link
0,"Após polêmica, Marine Le Pen diz que abomina n...",A candidata da direita nacionalista à Presidên...,2017-04-28,mundo,NaN,http://www1.folha.uol.com.br/mundo/2017/04/187...
1,"Macron e Le Pen vão ao 2º turno na França, em ...",O centrista independente Emmanuel Macron e a d...,2017-04-23,mundo,NaN,http://www1.folha.uol.com.br/mundo/2017/04/187...
2,"Apesar de larga vitória nas legislativas, Macr...",As eleições legislativas deste domingo (19) na...,2017-06-19,mundo,NaN,http://www1.folha.uol.com.br/mundo/2017/06/189...
3,"Governo antecipa balanço, e Alckmin anuncia qu...",O número de ocorrências de homicídios dolosos ...,2015-07-24,cotidiano,NaN,http://www1.folha.uol.com.br/cotidiano/2015/07...
4,"Após queda em maio, a atividade econômica sobe...","A economia cresceu 0,25% no segundo trimestre,...",2017-08-17,mercado,NaN,http://www1.folha.uol.com.br/mercado/2017/08/1...


In [14]:
from sklearn.model_selection import train_test_split

noticias_treino, noticias_teste = train_test_split(noticias_dataset,
                                                   test_size=0.2,
                                                   random_state=42)

In [15]:
noticias_treino.head()

,title,text,date,category,subcategory,link
11894,Justiça condena apenas um PM pela morte de jov...,A Justiça de São Paulo condenou um dos três po...,2017-03-16,cotidiano,NaN,http://www1.folha.uol.com.br/cotidiano/2017/03...
86291,O clássico da América,"OS DOIS times de maior torcida do Brasil, e do...",2015-12-07,colunas,jucakfouri,http://www1.folha.uol.com.br/colunas/jucakfour...
36859,Temer discute regra de transição para imposto ...,O presidente Michel Temer discute a adoção de ...,2017-06-13,mercado,NaN,http://www1.folha.uol.com.br/mercado/2017/06/1...
17897,"Além de SP, sete capitais vão aumentar a tarif...","Além de São Paulo, mais sete capitais brasilei...",2015-12-30,cotidiano,NaN,http://www1.folha.uol.com.br/cotidiano/2015/12...
81293,Professora da PUC aceita convite de Temer para...,A procuradora e professora Flavia Piovesan ace...,2016-05-17,colunas,monicabergamo,http://www1.folha.uol.com.br/colunas/monicaber...


In [16]:
len(noticias_treino)

88410

In [17]:
len(noticias_teste)

22103

In [18]:
noticias_treino.iloc[0]

title          Justiça condena apenas um PM pela morte de jov...
text           A Justiça de São Paulo condenou um dos três po...
date                                                  2017-03-16
category                                               cotidiano
subcategory                                                  NaN
link           http://www1.folha.uol.com.br/cotidiano/2017/03...
Name: 11894, dtype: object

In [19]:
noticias_treino.iloc[0].text

'A Justiça de São Paulo condenou um dos três policiais militares pela morte de Paulo Henrique Porto de Oliveira, 18, morto após se entregar depois de um assalto na zona oeste da cidade, em 2015.  O policial Tyson Oliveira Bastiane foi condenado a 12 anos, 5 meses e 17 dias de prisão, em regime fechado, por homicídio simples. Bastiane também cumprirá pena de 7 meses e 15 dias de detenção, em regime aberto, por fraude processual, além do pagamento de 35 dias-multa.  Já Silvano Clayton dos Reis foi absolvido da acusação de homicídio, mas condenado pelos crimes de falsidade ideológica e porte ilegal de arma a 4 anos, 11 meses e 17 dias de prisão no regime semiaberto. A juíza Giovana Christina Colares, da 3ª Vara do Júri, decidiu que o policial poderá recorrer em liberdade e expediu o alvará de soltura.  Segundo a denúncia, Bastiane foi o responsável por efetuar os tiros que matou o jovem de 18 anos. O advogado de Bastiane, que também representa o policial Reis, afirmou que irá recorrer da 

In [20]:
noticias_treino.iloc[0].title

'Justiça condena apenas um PM pela morte de jovem após assalto em 2015'

# Tema 2 - Computar vetores para comparar e encontrar conceitos

## Reusando modelos Word2vec

In [21]:
from gensim.models import KeyedVectors

modelo = KeyedVectors.load_word2vec_format("cbow_s300.txt")


In [22]:
modelo.get_vector('cachorro')

array([-4.35630e-02, -3.28700e-02,  1.52735e-01, -1.56780e-02,
        1.21885e-01, -1.67688e-01, -1.02211e-01, -8.61910e-02,
        7.20490e-02, -9.83480e-02, -1.32587e-01, -5.61530e-02,
       -2.27587e-01, -4.27520e-02, -1.19845e-01,  9.87550e-02,
       -4.37438e-01, -3.22793e-01,  1.24080e-02,  4.43268e-01,
       -7.31800e-02,  6.97510e-02, -1.43725e-01,  5.14352e-01,
       -1.16800e-03, -1.34310e-02, -3.70520e-02, -8.89080e-02,
        1.96335e-01,  4.87740e-01,  2.35393e-01, -1.07754e-01,
        1.87188e-01,  2.02846e-01,  2.27027e-01, -2.00064e-01,
        1.33436e-01,  8.17870e-02, -8.45000e-02, -4.21550e-02,
        2.71060e-02,  2.91997e-01, -2.40926e-01,  2.06865e-01,
        4.74000e-04,  2.06211e-01,  1.27639e-01,  3.85590e-02,
        1.77177e-01, -1.23831e-01, -8.69900e-02, -1.10168e-01,
       -1.31033e-01,  3.20510e-02, -7.07600e-03, -8.86480e-02,
        2.12310e-02,  2.30504e-01,  1.59084e-01,  3.89300e-02,
        4.15100e-03, -9.12450e-02, -1.90750e-02,  6.809

In [23]:
modelo.get_vector('gato')

array([ 0.080862, -0.115363,  0.375795,  0.114992, -0.036125, -0.090976,
       -0.202319, -0.297874,  0.063109, -0.322301, -0.311339, -0.145936,
        0.024642, -0.177522, -0.036947,  0.175441, -0.303248, -0.269762,
       -0.009697,  0.206333,  0.022104,  0.133072,  0.028073,  0.580374,
       -0.018641, -0.052175,  0.099459,  0.304987,  0.091809,  0.229643,
        0.187417, -0.129141, -0.06451 ,  0.096126,  0.118515, -0.040087,
       -0.019711, -0.078362, -0.078762,  0.326551,  0.163371,  0.074987,
       -0.07541 ,  0.178085, -0.129488,  0.334529,  0.02373 , -0.100327,
       -0.175369, -0.05598 ,  0.001715,  0.04005 , -0.187697,  0.057138,
        0.141195, -0.051365,  0.108957,  0.027061,  0.073435, -0.218011,
       -0.154617, -0.112563,  0.11286 ,  0.052005,  0.194585, -0.158321,
       -0.028461, -0.159067,  0.255324, -0.120326, -0.018783, -0.101366,
       -0.010514, -0.359594,  0.18899 ,  0.152701,  0.008957, -0.32782 ,
       -0.049077,  0.193421, -0.506094,  0.273253, 

## Testando o modelo Word2Vec

In [24]:
modelo.most_similar("cachorro")

[('cão', 0.7491682767868042),
 ('cachorrinho', 0.6738949418067932),
 ('gambá', 0.6491536498069763),
 ('cãozinho', 0.6424068212509155),
 ('cã£o', 0.6413822174072266),
 ('muffin', 0.6398880481719971),
 ('pônei', 0.6371211409568787),
 ('bichinho', 0.6330615282058716),
 ('filhote', 0.6309623718261719),
 ('garoto', 0.6273159384727478)]

In [25]:
modelo.most_similar("garoto")

[('rapaz', 0.8141030073165894),
 ('menino', 0.7310405373573303),
 ('bebê', 0.6767653226852417),
 ('homem', 0.6757827401161194),
 ('taxista', 0.6370103359222412),
 ('rapazinho', 0.6311320066452026),
 ('cachorro', 0.627315878868103),
 ('cãozinho', 0.6210364103317261),
 ('bandido', 0.6199325323104858),
 ('peixão', 0.6076808571815491)]

In [26]:
modelo.most_similar("china")

[('rússia', 0.7320704460144043),
 ('índia', 0.7241617441177368),
 ('tailândia', 0.701935887336731),
 ('indonésia', 0.6860769987106323),
 ('turquia', 0.6741335988044739),
 ('malásia', 0.6665689945220947),
 ('mongólia', 0.6593616008758545),
 ('manchúria', 0.6581847667694092),
 ('urss', 0.6581669449806213),
 ('grã-bretanha', 0.6568098068237305)]

In [27]:
modelo.most_similar("brasil")

[('japão', 0.7412911653518677),
 ('brasil.', 0.6550555229187012),
 ('pradã£o', 0.6525028347969055),
 ('haiti', 0.648073673248291),
 ('ocidente', 0.6387284994125366),
 ('trapichão', 0.6379075050354004),
 ('romildão', 0.6373966932296753),
 ('emtanto', 0.6351075172424316),
 ('romildã£o', 0.6259243488311768),
 ('beira-rio.', 0.6231665015220642)]

In [28]:
modelo.most_similar("brazil")

[('dependentemente', 0.3951590657234192),
 ('signs', 0.39377015829086304),
 ('fishes', 0.3812129497528076),
 ('tegridade', 0.3802288770675659),
 ('soulitary', 0.37817472219467163),
 ('formática', 0.3730659782886505),
 ('australia', 0.3718786835670471),
 ('verrem', 0.37113064527511597),
 ('snowed', 0.3711206912994385),
 ('excellence', 0.3696613311767578)]

In [29]:
modelo.most_similar("eua")

[('e.u.a', 0.772014856338501),
 ('estados-unidos', 0.7275917530059814),
 ('bálcãs', 0.6725860238075256),
 ('paísesbaixos', 0.661286473274231),
 ('trópicos', 0.6483936309814453),
 ('himalaias', 0.6465877294540405),
 ('pvd', 0.6319639682769775),
 ('países-baixos', 0.6244395971298218),
 ('hammers', 0.6219980120658875),
 ('musseques', 0.6150403618812561)]

In [30]:
modelo.most_similar(positive=["china","eua"])

[('rússia', 0.6043822169303894),
 ('e.u.a', 0.5732004642486572),
 ('estados-unidos', 0.5679851770401001),
 ('índia', 0.5590813159942627),
 ('turquia', 0.5520168542861938),
 ('bielorússia', 0.5510701537132263),
 ('tailândia', 0.547263503074646),
 ('bielorrússia', 0.5433691143989563),
 ('malásia', 0.5389045476913452),
 ('indonésia', 0.538504421710968)]

# Tema 3 - Entender o pré-processamento para usar modelos de representação Word2vec

## Relação entre palavras

In [31]:
modelo.most_similar(positive=["bandido","garoto"])

[('rapaz', 0.8167956471443176),
 ('homem', 0.7218790054321289),
 ('ladrão', 0.7216314077377319),
 ('menino', 0.6998820304870605),
 ('assaltante', 0.6800928711891174),
 ('taxista', 0.6627659797668457),
 ('assassino', 0.6417649984359741),
 ('pescador', 0.6383834481239319),
 ('mendigo', 0.6343545317649841),
 ('seqüestrador', 0.6332120895385742)]

In [32]:
modelo.most_similar(positive=["bandido","árvore"])

[('ladrão', 0.5338584780693054),
 ('ribanceira', 0.5245423316955566),
 ('armadilha', 0.5153564810752869),
 ('galho', 0.510231614112854),
 ('monstro', 0.505108654499054),
 ('garoto', 0.4991573095321655),
 ('malfeitor', 0.49658527970314026),
 ('corcova', 0.4949728846549988),
 ('bombonzinho', 0.4940298795700073),
 ('tijolada', 0.4939206838607788)]

In [33]:
modelo.most_similar(positive=["brasil","argentina"])

[('chile', 0.6781662702560425),
 ('peru', 0.6348033547401428),
 ('venezuela', 0.6273865103721619),
 ('equador', 0.6037014722824097),
 ('bolívia', 0.6017140746116638),
 ('haiti', 0.5993807315826416),
 ('méxico', 0.5962306261062622),
 ('paraguai', 0.5957703590393066),
 ('uruguai', 0.5903672575950623),
 ('japão', 0.5893509984016418)]

In [34]:
modelo.most_similar(positive=["brasil","china"])

[('japão', 0.7257888317108154),
 ('ocidente', 0.5933547019958496),
 ('canadá', 0.5921916961669922),
 ('austrália', 0.5688517093658447),
 ('tibete', 0.5675076246261597),
 ('haiti', 0.5656366348266602),
 ('irã', 0.5653263330459595),
 ('camboja', 0.5621732473373413),
 ('índia', 0.5570496320724487),
 ('rússia', 0.5542328357696533)]

## Combinação de vetores de palavras

In [35]:
modelo.most_similar(positive=["bandido","garoto"],negative=["cachorro"])

[('rapaz', 0.6294464468955994),
 ('assaltante', 0.5902887582778931),
 ('taxista', 0.5723674297332764),
 ('ladrão', 0.5506724119186401),
 ('homem', 0.5368587970733643),
 ('menino', 0.5134603977203369),
 ('caminhoneiro', 0.5123695135116577),
 ('adolescente', 0.5119109749794006),
 ('detento', 0.5060938596725464),
 ('mototaxista', 0.5034871101379395)]

In [36]:
modelo.most_similar(positive=["lobo","matilha"],negative=["cão"])

[('alcateia', 0.4463878571987152),
 ('chusma', 0.4014967978000641),
 ('festa-comício', 0.393410325050354),
 ('braça', 0.39301973581314087),
 ('plêiade', 0.3870777487754822),
 ('micro-rotura', 0.3837582767009735),
 ('megaestrutura', 0.37981319427490234),
 ('cadeira-de-rodas', 0.37967121601104736),
 ('sensaзгo', 0.3768736720085144),
 ('operação-relâmpago', 0.37636274099349976)]

In [37]:
modelo.most_similar(positive=["estrela","nuvens"],negative=["nuvem"])

[('estrelas', 0.5497429966926575),
 ('plêiades', 0.379197895526886),
 ('colinas', 0.3746805191040039),
 ('trovoadas', 0.373703271150589),
 ('sombras', 0.3734194040298462),
 ('pombas', 0.3726757764816284),
 ('corredoras', 0.3640727698802948),
 ('cigarras', 0.36065393686294556),
 ('galáxias', 0.35754913091659546),
 ('luas', 0.3575345277786255)]

In [38]:
modelo.most_similar(positive=["professor","mulher"],negative=["homem"])

[('professora', 0.6192208528518677),
 ('aluna', 0.5449554324150085),
 ('esposa', 0.4978231191635132),
 ('ex-aluna', 0.4884248375892639),
 ('namorada', 0.4737858772277832),
 ('enfermeira', 0.4728144109249115),
 ('filha', 0.467373788356781),
 ('irmã', 0.45845913887023926),
 ('ex-namorada', 0.45824766159057617),
 ('ex-professora', 0.4510470926761627)]

In [39]:
modelo.most_similar(positive=["médico","mulher"],negative=["homem"])

[('enfermeira', 0.6180862188339233),
 ('psicóloga', 0.47447532415390015),
 ('dama-de-companhia', 0.4738234877586365),
 ('esposa', 0.46559134125709534),
 ('parteira', 0.46362876892089844),
 ('mãe', 0.4581768214702606),
 ('governanta', 0.45722925662994385),
 ('madrasta', 0.4569782614707947),
 ('menina', 0.44435906410217285),
 ('filha', 0.44341108202934265)]

In [40]:
modelo.most_similar(positive=["professora", "homem"], negative=["mulher"])

[('estudante', 0.5248496532440186),
 ('aluno', 0.515190601348877),
 ('rapaz', 0.4828214645385742),
 ('professor', 0.47735464572906494),
 ('detento', 0.4750629961490631),
 ('escriptor', 0.47291240096092224),
 ('ortoéster', 0.4547411799430847),
 ('garoto', 0.4507240653038025),
 ('mestre-escola', 0.4486427903175354),
 ('jardineiro', 0.44736987352371216)]

# Tema 4 - Aprender a classificar com Word2vec

Classificando com Word2vec

In [41]:
noticias_treino.head()

,title,text,date,category,subcategory,link
11894,Justiça condena apenas um PM pela morte de jov...,A Justiça de São Paulo condenou um dos três po...,2017-03-16,cotidiano,NaN,http://www1.folha.uol.com.br/cotidiano/2017/03...
86291,O clássico da América,"OS DOIS times de maior torcida do Brasil, e do...",2015-12-07,colunas,jucakfouri,http://www1.folha.uol.com.br/colunas/jucakfour...
36859,Temer discute regra de transição para imposto ...,O presidente Michel Temer discute a adoção de ...,2017-06-13,mercado,NaN,http://www1.folha.uol.com.br/mercado/2017/06/1...
17897,"Além de SP, sete capitais vão aumentar a tarif...","Além de São Paulo, mais sete capitais brasilei...",2015-12-30,cotidiano,NaN,http://www1.folha.uol.com.br/cotidiano/2015/12...
81293,Professora da PUC aceita convite de Temer para...,A procuradora e professora Flavia Piovesan ace...,2016-05-17,colunas,monicabergamo,http://www1.folha.uol.com.br/colunas/monicaber...


In [42]:
noticias_treino.title.iloc[11]

'Uma pitanga na Paulista'

In [43]:
noticias_treino.title.iloc[12]

"Tite se diz surpreso com resultado e admite 'fase complicada'"

In [44]:
import string
import nltk
nltk.download("punkt")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [45]:
def tratarTokenizar(texto):
  texto=texto.lower()
  tokens_validos = []

  for token in nltk.word_tokenize(texto):
    if token not in string.punctuation:
      tokens_validos.append(token)

  return tokens_validos

In [46]:
tratarTokenizar("Estamos conseguindo, muito bem!")

['estamos', 'conseguindo', 'muito', 'bem']

In [47]:
import numpy as np

def somaVetores(tokens):
  vetorSomado = np.zeros(300)
  for token in tokens:
    vetorSomado += modelo.get_vector(token)

  return vetorSomado

In [48]:
tokens = tratarTokenizar("Estamos conseguindo, muito bem!")
texto_vetorizado = somaVetores(tokens)
texto_vetorizado

array([ 1.90810040e-02,  5.79588030e-01,  3.92183997e-01,  2.70103006e-01,
       -2.17770040e-02, -1.72040008e-01, -3.14922994e-01,  7.12581996e-01,
        2.98520006e-01, -4.36208995e-01,  2.57048003e-01, -5.79062020e-01,
       -4.36612010e-01,  3.37754991e-01,  5.34636984e-01,  3.70642988e-01,
       -5.77713989e-01, -3.55541995e-01, -1.78312998e-01, -7.03850109e-02,
        9.96995717e-04, -2.31660005e-01, -1.70079945e-02, -6.67464992e-01,
        2.52241980e-01,  2.11931000e-01,  3.05566002e-01,  2.22600102e-02,
       -2.54378016e-01,  4.22801986e-01, -1.22830003e-01,  3.95999998e-02,
       -1.96435995e-01,  4.38996987e-01,  2.47272011e-01,  9.73040070e-02,
        5.00208986e-01, -3.83796006e-01,  1.05107997e-01,  3.24586004e-01,
        7.11110011e-02,  2.29026988e-01,  2.75413003e-01,  5.20681992e-01,
       -1.50244996e-01,  9.40819979e-02,  6.22420003e-01,  2.70574003e-01,
       -2.32953008e-01,  2.38589987e-01, -1.42603002e-01, -2.51709998e-01,
        1.16967001e-01,  

In [49]:
len(texto_vetorizado)

300

In [50]:
#tokens = tratarTokenizar("Estamos conseguindo, muito bem! palavraEstranha")
#texto_vetorizado = somaVetores(tokens)
#texto_vetorizado

KeyError: ignored

In [51]:
#tokens = tratarTokenizar("Estamos conseguindo, muito bem! Somo nota 10")
#texto_vetorizado = somaVetores(tokens)
#texto_vetorizado

KeyError: ignored

In [52]:
import numpy as np

def somaVetores(tokens):
  vetorSomado = np.zeros(300)
  for token in tokens:
    try:
      vetorSomado += modelo.get_vector(token)
    except:
      if token.isnumeric():
        token = "0"*len(token)
        vetorSomado += modelo.get_vector(token)
      else:
        vetorSomado += modelo.get_vector("unknown")

  return vetorSomado

In [53]:
tokens = tratarTokenizar("Estamos conseguindo, muito bem! palavraEstranha")
texto_vetorizado = somaVetores(tokens)
texto_vetorizado

array([ 3.73647012e-01,  5.24777029e-01,  3.53743996e-01,  3.94161006e-01,
       -1.02413006e-01, -1.12844009e-01, -2.94502994e-01,  5.33922989e-01,
        1.17185004e-01, -5.27326996e-01,  2.11288005e-01, -6.85189022e-01,
       -8.38348001e-01,  7.57739004e-01,  5.01311982e-01,  5.48347993e-01,
       -1.12110098e+00, -4.40732991e-01, -7.11546998e-01, -3.56483998e-01,
        3.43599953e-02, -4.17471003e-01,  5.17180087e-02, -1.47285005e-01,
       -4.61600311e-02,  1.79560998e-01,  3.30969878e-02, -7.69669861e-02,
       -5.74660106e-02,  2.40588993e-01, -4.30904001e-01,  2.63600051e-03,
       -1.06038995e-01,  4.79484988e-01,  1.82597008e-01,  1.41078008e-01,
        5.89552989e-01, -4.33634005e-01,  2.42772002e-01,  1.96837008e-01,
        5.97040011e-02,  1.24347985e-01,  1.36222001e-01,  5.15467992e-01,
        1.40620992e-01, -1.94100291e-03,  8.61156006e-01,  5.97077012e-01,
        9.71269943e-02,  4.42208990e-01, -1.32925002e-01, -1.32709995e-01,
        9.90360014e-02,  

In [54]:
tokens = tratarTokenizar("Estamos conseguindo, muito bem! Somos nota 10")
texto_vetorizado = somaVetores(tokens)
texto_vetorizado

array([-4.97799944e-02,  8.22999025e-01,  1.49657991e-01,  5.84910015e-01,
       -1.41880065e-02, -2.99947013e-01, -4.87201475e-03,  1.39556300e+00,
        7.18258999e-01, -2.30644995e-01,  5.64835988e-01, -9.46917018e-01,
       -1.36112012e-01,  5.45287997e-01,  8.45332977e-01,  9.44199953e-02,
        7.73300976e-03, -1.08947298e+00, -2.62504993e-01, -4.11050016e-01,
       -2.30097011e-01, -1.53046003e-01,  9.73110134e-02, -8.92669987e-01,
        2.85330981e-01, -7.96150025e-02, -4.03410021e-01,  7.93370083e-02,
       -5.58039020e-01,  1.69111991e-01, -4.84905995e-01, -1.33749992e-02,
        2.52260003e-01,  2.19473993e-01,  5.55422001e-01, -1.30592997e-01,
        5.68335970e-01, -1.61892012e-01,  1.02460001e-01,  7.08263007e-01,
        3.00596002e-01, -1.32038016e-01,  5.48315004e-01,  1.13333500e+00,
        2.03250010e-01,  8.76590014e-02,  1.14962902e+00,  3.32706011e-01,
       -2.96181019e-01, -1.45990169e-02, -3.74589998e-01,  1.90500030e-02,
        2.98361988e-01,  

In [55]:
def vetoresTextos(textos):
  x = len(textos)
  y = 300
  vetores = np.zeros((x,y))

  for itexto in range(x):
    tokens = tratarTokenizar(textos.iloc[itexto])
    vetores[itexto] = somaVetores(tokens)

  return vetores

In [56]:
vetores_treino = vetoresTextos(noticias_treino.title)
vetores_teste = vetoresTextos(noticias_teste.title)
print(vetores_treino.shape)
print(vetores_teste.shape)

(88410, 300)
(22103, 300)


In [57]:
from sklearn.linear_model import LogisticRegression

classificador = LogisticRegression()

In [58]:
classificador.fit(vetores_treino, noticias_treino.category)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [59]:
classificador.score(vetores_teste, noticias_teste.category)

0.802832194724698

In [60]:
classificador = LogisticRegression(max_iter=200)

In [61]:
classificador.fit(vetores_treino, noticias_treino.category)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=200,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [62]:
classificador.score(vetores_teste, noticias_teste.category)

0.8027869519974664

## Interpretando a classificação 

In [63]:
noticias_treino.category.unique()

array(['cotidiano', 'colunas', 'mercado', 'esporte', 'mundo', 'ilustrada'],
      dtype=object)

In [64]:
categorias_previstas = classificador.predict(vetores_teste)
categorias_previstas

array(['cotidiano', 'mercado', 'mercado', ..., 'ilustrada', 'cotidiano',
       'cotidiano'], dtype=object)

In [65]:
len(categorias_previstas)

22103

In [66]:
categorias_previstas[0:20]

array(['cotidiano', 'mercado', 'mercado', 'colunas', 'mercado', 'esporte',
       'esporte', 'esporte', 'ilustrada', 'esporte', 'mundo', 'esporte',
       'mundo', 'esporte', 'esporte', 'mundo', 'colunas', 'mundo',
       'mercado', 'esporte'], dtype=object)

In [67]:
noticias_teste.category.iloc[0:20]

4129        esporte
104176    cotidiano
48253       mercado
14221       mercado
82528       mercado
57490       esporte
31203       esporte
85739       esporte
50981     ilustrada
10034         mundo
4081          mundo
11216       esporte
186           mundo
93513       esporte
35121       esporte
8663          mundo
110276      colunas
9451        colunas
6152        mercado
81066       esporte
Name: category, dtype: object

In [68]:
noticias_teste.iloc[0].title

'Prefeito do Rio alerta para risco de atraso na obra do metrô para Jogos'

In [69]:
noticias_teste.iloc[1].title

'Após embate com associações médicas, governo recua e vai revisar decreto'

## Comparando Classificadores

In [70]:
from sklearn.metrics import classification_report

categorias_previstas = classificador.predict(vetores_teste)
relatorio = classification_report(noticias_teste.category, categorias_previstas)
print(relatorio)

              precision    recall  f1-score   support

     colunas       0.75      0.73      0.74      4142
   cotidiano       0.78      0.79      0.78      3440
     esporte       0.90      0.89      0.89      3991
   ilustrada       0.79      0.80      0.79      2935
     mercado       0.78      0.79      0.79      4149
       mundo       0.82      0.83      0.82      3446

    accuracy                           0.80     22103
   macro avg       0.80      0.80      0.80     22103
weighted avg       0.80      0.80      0.80     22103



In [71]:
vetores_treino_corpo = vetoresTextos(noticias_treino.text)
vetores_teste_corpo = vetoresTextos(noticias_teste.text)
print(vetores_treino_corpo.shape)
print(vetores_teste_corpo.shape)

(88410, 300)
(22103, 300)


In [72]:
classificador_corpo = LogisticRegression(max_iter=200)

In [73]:
classificador_corpo.fit(vetores_treino_corpo, noticias_treino.category)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=200,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [74]:
categorias_previstas_corpo = classificador_corpo.predict(vetores_teste_corpo)
relatorio_classificador_corpo = classification_report(noticias_teste.category, categorias_previstas_corpo)
print(relatorio_classificador_corpo)

              precision    recall  f1-score   support

     colunas       0.77      0.73      0.75      4142
   cotidiano       0.86      0.87      0.86      3440
     esporte       0.94      0.95      0.95      3991
   ilustrada       0.84      0.85      0.84      2935
     mercado       0.84      0.85      0.85      4149
       mundo       0.89      0.91      0.90      3446

    accuracy                           0.86     22103
   macro avg       0.86      0.86      0.86     22103
weighted avg       0.86      0.86      0.86     22103



# Tema 5 - Avaliar a classificação com Word2vec CBOW e Skip-gram

## Classificando com Skip-gram

In [75]:
!unzip "/content/drive/My Drive/Unyleya - Mineração de Texto e PLN/skip_s300.zip"

Archive:  /content/drive/My Drive/Unyleya - Mineração de Texto e PLN/skip_s300.zip
  inflating: skip_s300.txt           


In [76]:
modelo_skipgram = KeyedVectors.load_word2vec_format("skip_s300.txt")

In [77]:
import numpy as np

def somaVetores(tokens, modelo):
  vetorSomado = np.zeros(300)
  for token in tokens:
    try:
      vetorSomado += modelo.get_vector(token)
    except:
      if token.isnumeric():
        token = "0"*len(token)
        vetorSomado += modelo.get_vector(token)
      else:
        vetorSomado += modelo.get_vector("unknown")

  return vetorSomado

In [79]:
def vetoresTextos(textos, modelo):
  x = len(textos)
  y = 300
  vetores = np.zeros((x,y))

  for itexto in range(x):
    tokens = tratarTokenizar(textos.iloc[itexto])
    vetores[itexto] = somaVetores(tokens, modelo)

  return vetores

In [80]:
vetores_treino_skipgram = vetoresTextos(noticias_treino.title, modelo_skipgram)
vetores_teste_skipgram = vetoresTextos(noticias_teste.title, modelo_skipgram)

In [81]:
classificador_skipgram = LogisticRegression(max_iter=200)
classificador_skipgram.fit(vetores_treino_skipgram, noticias_treino.category)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=200,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [82]:
categorias_previstas_skipgram = classificador_skipgram.predict(vetores_teste_skipgram)
relatorio_skipgram = classification_report(noticias_teste.category, categorias_previstas_skipgram)
print(relatorio_skipgram)

              precision    recall  f1-score   support

     colunas       0.75      0.73      0.74      4142
   cotidiano       0.81      0.81      0.81      3440
     esporte       0.91      0.90      0.91      3991
   ilustrada       0.81      0.82      0.81      2935
     mercado       0.79      0.81      0.80      4149
       mundo       0.84      0.85      0.84      3446

    accuracy                           0.82     22103
   macro avg       0.82      0.82      0.82     22103
weighted avg       0.82      0.82      0.82     22103



In [83]:
print(relatorio) # classificacao de titulos com cbow

              precision    recall  f1-score   support

     colunas       0.75      0.73      0.74      4142
   cotidiano       0.78      0.79      0.78      3440
     esporte       0.90      0.89      0.89      3991
   ilustrada       0.79      0.80      0.79      2935
     mercado       0.78      0.79      0.79      4149
       mundo       0.82      0.83      0.82      3446

    accuracy                           0.80     22103
   macro avg       0.80      0.80      0.80     22103
weighted avg       0.80      0.80      0.80     22103



## Comparando Skip-gram e CBOW

In [84]:
vetores_treino_skipgram_corpo = vetoresTextos(noticias_treino.text, modelo_skipgram)
vetores_teste_skipgram_corpo = vetoresTextos(noticias_teste.text, modelo_skipgram)

In [85]:
classificador_skipgram_corpo = LogisticRegression(max_iter=200)
classificador_skipgram_corpo.fit(vetores_treino_skipgram_corpo, noticias_treino.category)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=200,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [86]:
categorias_previstas_skipgram_corpo = classificador_skipgram_corpo.predict(vetores_teste_skipgram_corpo)
relatorio_skipgram_corpo = classification_report(noticias_teste.category, categorias_previstas_skipgram_corpo)
print(relatorio_skipgram_corpo)

              precision    recall  f1-score   support

     colunas       0.77      0.74      0.75      4142
   cotidiano       0.87      0.88      0.87      3440
     esporte       0.95      0.96      0.96      3991
   ilustrada       0.85      0.86      0.85      2935
     mercado       0.86      0.86      0.86      4149
       mundo       0.90      0.91      0.91      3446

    accuracy                           0.87     22103
   macro avg       0.87      0.87      0.87     22103
weighted avg       0.87      0.87      0.87     22103



In [87]:
print(relatorio_classificador_corpo) #classificador usando corpo e cbow

              precision    recall  f1-score   support

     colunas       0.77      0.73      0.75      4142
   cotidiano       0.86      0.87      0.86      3440
     esporte       0.94      0.95      0.95      3991
   ilustrada       0.84      0.85      0.84      2935
     mercado       0.84      0.85      0.85      4149
       mundo       0.89      0.91      0.90      3446

    accuracy                           0.86     22103
   macro avg       0.86      0.86      0.86     22103
weighted avg       0.86      0.86      0.86     22103



In [89]:
noticias_treino.query("category == 'colunas'")

,title,text,date,category,subcategory,link
86291,O clássico da América,"OS DOIS times de maior torcida do Brasil, e do...",2015-12-07,colunas,jucakfouri,http://www1.folha.uol.com.br/colunas/jucakfour...
81293,Professora da PUC aceita convite de Temer para...,A procuradora e professora Flavia Piovesan ace...,2016-05-17,colunas,monicabergamo,http://www1.folha.uol.com.br/colunas/monicaber...
83333,"Cidade linda, sem catadores, é lixo",A frase acima virou slogan dos catadores de ma...,2017-07-04,colunas,maragama,http://www1.folha.uol.com.br/colunas/maragama/...
92462,Raiva contida,"Um psicanalista carioca escreveu que, se o Bra...",2015-10-06,colunas,franciscodaudt,http://www1.folha.uol.com.br/colunas/francisco...
66298,Uma pitanga na Paulista,Coloquei o pé pra fora do táxi e quase pisei n...,2015-05-24,colunas,denisefraga,http://www1.folha.uol.com.br/colunas/denisefra...
...,...,...,...,...,...,...
18431,Janaína Paschoal é hostilizada em aeroporto de...,Um vídeo que circula nas redes sociais desde a...,2016-06-30,colunas,monicabergamo,http://www1.folha.uol.com.br/colunas/monicaber...
5311,Pressionar crianças para que sejam bons alunos...,Pesquisas apontam aumento na taxa de suicídio ...,2016-08-03,colunas,roselysayao,http://www1.folha.uol.com.br/colunas/roselysay...
67969,Ainda a poupança,"Meu colega Marcelo Miterhof, que ocupa este es...",2015-02-15,colunas,samuelpessoa,http://www1.folha.uol.com.br/colunas/samuelpes...
82386,Lula pede que Vicente Cândido retire 'emenda L...,Lula pediu ao deputado Vicente Cândido (PT-SP)...,2017-01-08,colunas,monicabergamo,http://www1.folha.uol.com.br/colunas/monicaber...


In [90]:
noticias_treino.head()

,title,text,date,category,subcategory,link
11894,Justiça condena apenas um PM pela morte de jov...,A Justiça de São Paulo condenou um dos três po...,2017-03-16,cotidiano,NaN,http://www1.folha.uol.com.br/cotidiano/2017/03...
86291,O clássico da América,"OS DOIS times de maior torcida do Brasil, e do...",2015-12-07,colunas,jucakfouri,http://www1.folha.uol.com.br/colunas/jucakfour...
36859,Temer discute regra de transição para imposto ...,O presidente Michel Temer discute a adoção de ...,2017-06-13,mercado,NaN,http://www1.folha.uol.com.br/mercado/2017/06/1...
17897,"Além de SP, sete capitais vão aumentar a tarif...","Além de São Paulo, mais sete capitais brasilei...",2015-12-30,cotidiano,NaN,http://www1.folha.uol.com.br/cotidiano/2015/12...
81293,Professora da PUC aceita convite de Temer para...,A procuradora e professora Flavia Piovesan ace...,2016-05-17,colunas,monicabergamo,http://www1.folha.uol.com.br/colunas/monicaber...
